# Part 2: Regression

Please include any imports (allowed by Ed) you require throughout your notebook in the first cell.

In [287]:
# Import all required libraries
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib as mp
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import RidgeCV,LassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier


## Data loading and preprocessing
The data file, called `concrete.csv`, is loaded into the same directory as the notebook you are working in. 

You will need to load this file into numpy arrays for the attribute data and the labels. So that we can test your code more effectively, please complete this task inside the given function scaffold, and have your function return these arrays.

Begin by looking at the file and its format. Notice any missing values and how they are encoded in the file. In the returned X array, missing values should be encoded as [np.nan](https://numpy.org/doc/stable/user/misc.html).

While there are multiple ways to load the file correctly, a suggested function to use is [`pd.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). Look through the documentation to check which arguments you will need to pass to the function to load the file correctly. If you choose to use this approach, you will need to extract the appropriate numpy arrays from the pandas dataframe, and exclude any headers. 

The `X` array returned by your function should have shape `(number of examples, number of attributes)`, and the `y` array returned by your function should have shape `(number of examples,)`. We will also test your function with some different datasets with the same data types, delimiters, and encoding of missing values. However, these files may have a different filename, number of examples and/or attributes, so you should not hard code these values in your solution. There will not be any missing target values, and the targets will always be in the final column. 



In [288]:
### TEST FUNCTION: test_data_loading
def load_concrete_data(filename):
    """Load the dataset located at the filname string as described above."""
    attribute_array = np.array([])
    target_array = np.array([])
    data=pd.read_csv(filename,sep=";", na_values="?")
    attribute_array= data.iloc[:,:-1].values.astype(np.float64) #extracting all attributes except the target data column
    target_array= data.iloc[:, -1].values.astype(np.float64) # target data column
    return attribute_array, target_array

In [289]:
### SKIP
# This cell won't be marked. Use it to try out your code.
def load_concrete_data(filename):
    """Load the dataset located at the filname string as described above."""
    attribute_array = [],
    target_array =[]
    data=pd.read_csv(filename,sep=";", na_values="?")
    attribute_array= data.iloc[:,:-1].values.astype(np.float64) #extracting all attributes except the target data column
    target_array= data.iloc[:, -1].values.astype(np.float64) # target data column
    return attribute_array, target_array
    
attribute_array, target_array = load_concrete_data('concrete.csv')
print(attribute_array[:5], target_array[:5])


[[ 540.     0.     0.   162.     2.5 1040.     nan   28. ]
 [ 300.     0.     0.   184.     0.  1075.   795.     7. ]
 [ 251.8    0.    99.9  146.1   12.4 1006.   899.8   56. ]
 [ 203.5  305.3    0.   203.5    0.   963.4  630.     7. ]
 [ 149.   236.     0.   176.    13.   847.   893.    28. ]] [79.99 15.58 44.14 19.54 32.96]


The next job is to deal with any missing values in the data. Complete the function below which should replace any missing values with the mean value for that attribute. Take a look at the following documentation to see a suitable function to perform this using sklearn: [Documentation](https://scikit-learn.org/stable/modules/impute.html#impute).

In [290]:
### TEST FUNCTION: test_missing_vals
def fill_missing_values(X):
    """Fill missing (np.nan) values in the input array as described above."""
    missing_field = SimpleImputer (missing_values=np.nan, strategy='mean')
    data_new = missing_field.fit_transform(X)
    return data_new

In [291]:
### SKIP
# This cell won't be marked. Use it to try out your code.
def fill_missing_values(X):
    """Fill missing (np.nan) values in the input array as described above."""
    missing_field = SimpleImputer (missing_values=np.nan, strategy='mean')
    data_new = missing_field.fit_transform(X)
    return data_new
attribute_new_array = fill_missing_values(attribute_array)
print(attribute_new_array[:5], target_array[:5])

[[ 540.            0.            0.          162.            2.5
  1040.          775.20388514   28.        ]
 [ 300.            0.            0.          184.            0.
  1075.          795.            7.        ]
 [ 251.8           0.           99.9         146.1          12.4
  1006.          899.8          56.        ]
 [ 203.5         305.3           0.          203.5           0.
   963.4         630.            7.        ]
 [ 149.          236.            0.          176.           13.
   847.          893.           28.        ]] [79.99 15.58 44.14 19.54 32.96]


Many of the algorithms will benefit from scaling/transforming the features to a similar range. In Part 1 of the assignment and the labs thus far, we have used min-max scaling to achieve this. Here we would like to utilise a different approach (see the related question in the quiz component).

There are various other options implemented in scikit-learn (you might find [this documentation](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html) useful to understand the difference between them). Try utilising [`sklearn.preprocessing.PowerTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html#sklearn.preprocessing.PowerTransformer) with the 'yeo-johnson' transformation to transform the data so that features have a zero mean and variance of 1.

In [292]:
### TEST FUNCTION: test_scaling
def scale_data(X):
    """Scale data using PowerTransformer as described above."""
    transformer = PowerTransformer(method='yeo-johnson', standardize=True)
    attribute_transformed = transformer.fit_transform(X)

    return attribute_transformed

In [293]:
### SKIP
# This cell won't be marked. Use it to try out your code.

Now you are ready to load and preprocess the dataset. Load the `concrete.csv` file and preprocess it using the functions above. Name your resulting numpy arrays `X` and `y` for the attribute data and targets respectively.

In [294]:
### TEST FUNCTION: test_preprocessing_usage
# We will check the values of variables X and y are correct
attribute_array, target_array = load_concrete_data('concrete.csv')
attribute_new_array = fill_missing_values(attribute_array)
X = scale_data(attribute_new_array)
y = target_array

In [295]:
### TEST FUNCTION: test_preprocessing_hidden
# Do not use or delete this cell.
# Your functions above will be tested here with several datasets
# different to the ones given. If you are failing test_preprocessing_hidden,
# make sure your functions work with other datasets as outlined above.

## Implementing algorithms and evaluating their performance with cross validation

In the following components, you are required to implement functions which create algorithms and evaluate them with 10 fold cross validation. 

In order to make this reproducible, it is important that the folds are kept consistent across runs. You can utilise `sklearn.model_selection.KFold` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)) to provide consistency in the generation of the 10 folds. This object can be passed into the `cv` argument of `cross_val_score`. Make sure to shuffle the data in this process, and set `random_state` to be 0. Use the variable name `cvKFold`.

In [296]:
### TEST FUNCTION: test_cvkfold

cvKFold = KFold(n_splits=10, shuffle=True, random_state=0)
pipeline = make_pipeline(PowerTransformer(method='yeo-johnson', standardize=True),
    KNeighborsRegressor()
)
scores = cross_val_score(pipeline, X, y, cv=cvKFold)

### Weighted KNN Regression
We have seen how to implement a KNN classifier in the lab and first part of the assignment. Now your task is to implement a KNN for regression using `sklearn.neighbors.KNeighborsRegressor` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)).

Use the function definition below, so that algorithm hyperparameters such as number of neighbors, power of minkowski distance, and weighting strategy can optionally be passed in and accessed as a dictionary. Note you also can pass arguments as a dictionary to functions (such as sklearn constructors) using the ** syntax. 

You should create an instance of the model using the supplied hyperparameters and run 10 fold cross validation (using the fold set up above and your preprocessed data X and y) to evaluate it. The function should return the model instance and the average cross validation R^2 score obtained. 

In [297]:
### TEST FUNCTION: test_knn
def evaluate_knn_regression(X, y, **hyperparams):
    cvKFold = KFold(n_splits=10, shuffle=True, random_state=0)
    knn_model = KNeighborsRegressor(**hyperparams)
    r2 = cross_val_score(knn_model, X, y, cv=cvKFold, scoring='r2')
    mean_r2_score = np.mean(r2)
    return knn_model, mean_r2_score

In [298]:
### SKIP
# Try out your function here - this code won't be marked
def evaluate_knn_regression(X, y, **hyperparams):
    cvKFold = KFold(n_splits=10, shuffle=True, random_state=0)
    knn_model = KNeighborsRegressor(**hyperparams)
    scale_da = make_pipeline(PowerTransformer(method='yeo-johnson', standardize=True), knn_model)
    r2 = cross_val_score(knn_model, X, y, cv=cvKFold, scoring='r2')

    mean_r2_score = np.mean(r2)

    return knn_model, mean_r2_score

Now its time to try out your KNN regression using a weighted averaging strategy, which can often improve the performance. Use 9 nearest neighbours with the Euclidean distance. First print the score without weighting. Then print the score using a  weighting strategy based on the inverse of the distance.
The format of your output should be:
```
Cross validation R^2 score without weighting: x.xx
Cross validation R^2 score with weighting: x.xx
```

In [299]:
### TEST FUNCTION: test_weighted_knn_usage
hyperparams_knn = {'n_neighbors': 9, 'weights': 'uniform', 'p': 2}
knn_model_without_weighting, r2_score_without_weighting = evaluate_knn_regression(X, y, **hyperparams_knn)
knn_weighted_hyperparams = {'n_neighbors': 9, 'weights': 'distance', 'p': 2}
model_with_weighting, r2_score_with_weighting = evaluate_knn_regression(X, y, **knn_weighted_hyperparams)
print(f"Cross validation R^2 score without weighting: {r2_score_without_weighting:.2f}")
print(f"Cross validation R^2 score with weighting: {r2_score_with_weighting:.2f}")


Cross validation R^2 score without weighting: 0.77
Cross validation R^2 score with weighting: 0.81


### Linear regression
Implement similar functions to above to create Ridge and Lasso regression models and perform cross validation with them. They should return the model you have created and the cross validation R^2 score.

In [300]:
### TEST FUNCTION: test_ridge_regression
def evaluate_ridge(X, y, **hyperparams):
    ridge_model = Ridge(**hyperparams)
    scores = cross_val_score(ridge_model, X, y, cv=cvKFold, scoring='r2')
    mean_r2 = scores.mean()
    return ridge_model, mean_r2

In [301]:
### TEST FUNCTION: test_lasso_regression
def evaluate_lasso(X, y, **hyperparams):
    lasso_model = Lasso(**hyperparams)
    scores = cross_val_score(lasso_model, X, y, cv=cvKFold, scoring='r2')
    mean_score = scores.mean()
    return lasso_model, mean_score


In [302]:
### SKIP
# Try out your function here - this code won't be marked
# You can also use this cell to answer the related question in Part 2 Questions
# Assuming you have already loaded your data into X and y

Perform cross validation for ridge and lasso regression, both with alpha=10. Ensure to pass a `random_state` hyperparameter set to 0 for reproducibility. The format of your output should be:

```
Ridge cross validation R^2 score: x.xx
Lasso cross validation R^2 score: x.xx
```

In [303]:
### TEST FUNCTION: test_ridge_lasso_usage
ridgehyperparams = {'alpha': 10.0, 'random_state': 0}
lassohyperparams = {'alpha': 10.0,  'random_state': 0 }
ridge_model, ridge_r2_score = evaluate_ridge(X, y, **ridgehyperparams)
lasso_model, lasso_r2_score = evaluate_lasso(X, y, **lassohyperparams)

print(f"Ridge cross validation R^2 score: {ridge_r2_score:.2f}", )
print(f"Lasso cross validation R^2 score: {lasso_r2_score:.2f}", )

Ridge cross validation R^2 score: 0.76
Lasso cross validation R^2 score: -0.02


### Decision Trees
Decision trees can also be used for regression problems. However in this case we can't choose the best splits on the attribute values using entropy as this relied on looking at the distribution of classes across the resulting subsets of data. Several different criteria can be used to determine the best attribute split. Read through this [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) to take a look at these and understand the class you will be using.

In [304]:
### TEST FUNCTION: test_decision_tree
def evaluate_decision_tree(X, y, **hyperparams):
    regressor = DecisionTreeRegressor(**hyperparams)
    regressor.fit(X,y)
    score = cross_val_score(regressor, X, y, cv= cvKFold, scoring='r2')
    mean_r2 = score.mean()
    return regressor, mean_r2

In [305]:
### SKIP
# Try out your function here - this code won't be marked
X_train, X_test, y_train, y_test = train_test_split(X, y)
criterion = 'friedman_mse'
max_depth = 50
random_state =0

regressor = DecisionTreeRegressor(criterion=criterion, max_depth=max_depth, random_state=random_state)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

scores = cross_val_score(regressor, X, y, scoring='r2',error_score='raise')
mean_r2 = scores.mean()
print(scores, mean_r2)

[0.51079008 0.66957267 0.78828521 0.74504584 0.45295744] 0.6333302468905162


Test out your decision tree with the Friedman mean squared error criterion and a max_depth of 50. Make sure to set the `random_state` to 0 so that we can check your output. Print your output in the format:
```
Cross validation R^2 score: x.xx
```

In [306]:
### TEST FUNCTION: test_decision_tree_usage
decision_tree_hyperparams = {
    'criterion': 'friedman_mse',  
    'max_depth': 50,
    'random_state': 0
}

decision_tree_regressor, decision_tree_r2_score = evaluate_decision_tree(X, y, **decision_tree_hyperparams)
print(f"Cross validation R^2 score: {decision_tree_r2_score:.2f}")

Cross validation R^2 score: 0.70


## Ensemble Methods

---



---


### Random Forest
Building on the single decision tree, we can utilise the Random Forest ensemble method.

This time, your function that performs cross validation should take an extra argument to specify the metric to be used for the evaluation. See ([Model evaluation](https://scikit-learn.org/0.15/modules/model_evaluation.html)) and ([cross_val_score](https://scikit-learn.org/0.15/modules/model_evaluation.html)) for detail on how different metrics can be used in sklearn evaluations. During testing we will pass strings "`neg_mean_absolute_error`", "`neg_mean_squared_error`", and "`r2`" to specify the evaluation metric expected to be used. 


In [307]:
### TEST FUNCTION: test_random_forest
def evaluate_random_forest(X, y, metric=None, **hyperparams):
    regressor = RandomForestRegressor(**hyperparams)
    scores = cross_val_score(regressor, X, y, cv=cvKFold, scoring=metric)
    mean_score = scores.mean()
    return regressor, mean_score


In [308]:
### SKIP
# Try out your function here - this code won't be marked
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming you have your dataset loaded into X and y

# Define the Random Forest Regressor with specified hyperparameters
rf_regressor = RandomForestRegressor(
    criterion='friedman_mse',
    max_features=4,
    max_leaf_nodes=10,
    random_state=0  # Set a random state for reproducibility
)

# Define a list of evaluation metrics
evaluation_metrics = {
    'negative MAE': 'neg_mean_absolute_error',
    'negative MSE': 'neg_mean_squared_error',
    'R^2 score': 'r2'
}

# Perform cross-validation for each metric and print the results
for metric_name, metric_scoring in evaluation_metrics.items():
    scores = cross_val_score(rf_regressor, X, y, cv=3, scoring=metric_scoring)
    avg_score = -scores.mean() if 'negative' in metric_name else scores.mean()
    print(f'Cross validation {metric_name}: {avg_score:.2f}')


Cross validation negative MAE: 6.71
Cross validation negative MSE: 73.12
Cross validation R^2 score: 0.72


Perform cross validation of a random forest ensemble which uses the Friedman mean squared error as the criterion, considers a maximum of 4 features when looking for the best split, and grows trees to have a max of 10 leaf nodes.
Perform the evaluation 3 times, once using each of the metrics that can be passed to your evaluation function.
Your ouput should have the format:
```
Cross validation negative MAE: x.xx
Cross validation negative MSE: x.xx
Cross validation R^2 score: x.xx

```

In [309]:
### TEST FUNCTION: test_random_forest_usage
hyperparameters = {
    'criterion': 'friedman_mse',
    'max_features': 4,
    'max_leaf_nodes': 10,
    'random_state': 0  
}

evaluation_metrics = {
    'negative MAE': 'neg_mean_absolute_error',
    'negative MSE': 'neg_mean_squared_error',
    'R^2 score': 'r2'
}
for metric_name, metric_scoring in evaluation_metrics.items():
    _, avg_score = evaluate_random_forest(X, y, metric=metric_scoring, **hyperparameters)
    print(f'Cross validation {metric_name}: {avg_score:.2f}')


Cross validation negative MAE: -6.78
Cross validation negative MSE: -74.39
Cross validation R^2 score: 0.71


There are questions related to these metrics in "Part 2 Questions".

#

## Hyperparameter Tuning using Grid Search with Cross Validation
Many of the regression algorithms we have studied have several important hyperparameters that need to be tuned to obtain effective performance. One way we have showed you how to perform this is grid search with cross validation. For this part, you will need to perform grid search with cross validation for the gradient boosting regression algorithm.

We need to split our data into a training and test set to perform this procedure appropriately.

In [310]:
### TEST FUNCTION: test_assert_splitting
# These variables can be used when performing your grid search later.

# TODO: comment the next line out and uncomment the following line to create the 
# train test split after you've defined X and y earlier in the notebook
#X_train, X_test, y_train, y_test = None, None, None, None
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


### Gradient boosting
Write a method to perform a cross validation grid search ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)) to tune over hyperparameters for a gradient boosting model ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)). You should continue to use the CVKFold you defined above to set up the folds.

Your function should return the fitted sklearn CVGridSearch object itself, the best hyperparameter combination (out of those tuned over) as a dictionary (eg. `{"learning_rate": 0.1, "n_estimators": 100, ...}`) and the best cross validation score found during training. Ensure the best model returned as part of the CVGridSearch object has been trained on the entire training set.

In [311]:
### TEST FUNCTION: test_gradient_boosting
def best_gradient_boosting(X, y, param_grid, metric=None):
    grid_search = GridSearchCV(
        GradientBoostingRegressor(random_state=0),
        param_grid=param_grid,
        cv=cvKFold,
        scoring=metric if metric is not None else 'r2',
        return_train_score=True
    ).fit(X, y)
    best_params = grid_search.best_params_
    best_cv_score = grid_search.best_score_
    best_model = grid_search.best_estimator_    
    return grid_search, best_params, best_cv_score



In the following cell, you should define a grid with three values of learning rates and maximum number of leaf nodes, and the possible combinations for the criterion.

Using this parameter grid, run your cross validation grid search with R^2 as the metric and print the best hyperparameter combination and the best model's test set R^2 score. 

Use the variable names provided in the scaffold. We will ensure your results are reproduced and that you set reasonable parameter values to achieve a threshold R^2. 

Output format:
```
Best hyperparameter combination: {'param1': value, 'param2': value, ...}
Best model's test set R^2 score: x.xx
```

In [312]:
### TEST FUNCTION: test_parameter_tuning

param_grid = {
        'learning_rate': [0.01, 0.3, 0.5],  
        'max_leaf_nodes': [6, 8, 10],   
        'criterion': ['friedman_mse', 'squared_error']

}

best_model, best_params, best_cv_score = best_gradient_boosting(X_train, y_train, param_grid, metric=None)
y_pred = best_model.predict(X_test)

test_score = r2_score(y_test, y_pred)
print("Best hyperparameter combination:", best_params)
print("Best model's test set R^2 score: {:.2f}".format(test_score))

Best hyperparameter combination: {'criterion': 'friedman_mse', 'learning_rate': 0.3, 'max_leaf_nodes': 6}
Best model's test set R^2 score: 0.90
